# Лабораторная работа

Баллы за секции:
- *Задание 1* - 10 баллов
- *Задание 2* - 20 баллов
- *Задание 3* - 30 баллов
- **Итого** - 60 баллов

Оценка за лабу $ = \frac{score}{6}$

## Внимание! Attention! Achtung!

**Данная лабораторная работа является блокирующей**, т.е. если лабораторная работа не выполнена, то оценка за курс - неуд.

Лабораторная работа считается выполненной, если **за каждое из заданий получено не менее 2 баллов**.

## Формат сдачи лабораторной работы

Данная лабораторная работа сдается так же, как и проект - заполняете данный ноутбук, отправляете его через GitHub с указанием проверяющего.

**Ассистенты не будут запускать ячейки кода, а смотреть на выведенный результат и код. Если ячейка кода не запущена - балл не ставится, даже при правильно написанном коде.**

***(Комментарии что курс по Python легче чем по C++ учтены)***

>**Примечание**
>
>Данные задания вдохновлены предметом "Введение в анализ данных" 2 курса, по итогу которого идёт отбор на DS-поток. Если вы видите себя в смежных областях, советуем выполнить данную лабораторку по максимуму.

In [ ]:
# Всё необходимое для выполнения данной лабораторной работы!
import requests
from time import sleep

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Задание 1. Парсинг - 10 баллов

В этом задании Вам предстоит работать с API сайта <a target="_blank" href="hh.ru">hh.ru</a>. Документация доступна по <a target="_blank" href="https://github.com/hhru/api/blob/master/docs/vacancies.md#search">ссылке</a>.

<b><font color="red">Внимание!</font> При работе с API не забывайте делать паузы между запросами, иначе вас забанят. Просрочка дедлайна по этой причине не принимается!</b>

**Задача:**
1. выбрать 5 интересующих вас профессий (не смежных, т.е. Аналатик и Data Engineer не считается корректным)
2. затем получить доступные по данным профессиям вакансии в Москве и выделить их физические координаты
3. нанести полученные координаты на карту местоположения компаний

*Пример запроса для профессии:*

In [ ]:
URL = 'https://api.hh.ru/vacancies'

params = {
    'text': "Физик-ядерщик",
    'area': 1,
    'page': 0,
    'per_page': 10
}

data = requests.get(URL, params).json()

Если Ваш запрос прошел успешно, то вы получите следующие ключи (значения данных ключей вы найдете по ссылке на документацию):

In [ ]:
data.keys()

dict_keys(['items', 'found', 'pages', 'page', 'per_page', 'clusters', 'arguments', 'fixes', 'suggests', 'alternate_url'])

>**Примечание**
>
>В `data['pages']` будет хранится количество найденных страниц вакансий - для корректного сбора данных Вам нужно пробежать по **КАЖДОЙ** странице и сделать соотвествующие запросы.

>**Совет**
>
>Посмотрите что хранится в `items`!

(1 балл) Выберете для нас 5 наиболее интересных специальностей и получите по ним имеющиеся вакансии:

In [ ]:
URL = 'https://api.hh.ru/vacancies'

names=["Python-разработчик", "C++-разработчик", "Data scientist", "Product manager", "Full-stack разработчик"]
data=[]
for name in names:
  params = {
    'text': name,
    'area': 1,
    'page':0,
    'per_page': 50

  }
  pages=requests.get(URL, params).json()["pages"]
  for i in range(pages):
    params["page"]=i
    response=requests.get(URL,params).json()
    for h in response["items"]:
      data.append(h)
    sleep(0.05)
    print(f"Parsed page {i} of {pages} for {name}")


Parsed page 0 of 11 for Python-разработчик
Parsed page 1 of 11 for Python-разработчик
Parsed page 2 of 11 for Python-разработчик
Parsed page 3 of 11 for Python-разработчик
Parsed page 4 of 11 for Python-разработчик
Parsed page 5 of 11 for Python-разработчик
Parsed page 6 of 11 for Python-разработчик
Parsed page 7 of 11 for Python-разработчик
Parsed page 8 of 11 for Python-разработчик
Parsed page 9 of 11 for Python-разработчик
Parsed page 10 of 11 for Python-разработчик
Parsed page 0 of 6 for C++-разработчик
Parsed page 1 of 6 for C++-разработчик
Parsed page 2 of 6 for C++-разработчик
Parsed page 3 of 6 for C++-разработчик
Parsed page 4 of 6 for C++-разработчик
Parsed page 5 of 6 for C++-разработчик
Parsed page 0 of 8 for Data scientist
Parsed page 1 of 8 for Data scientist
Parsed page 2 of 8 for Data scientist
Parsed page 3 of 8 for Data scientist
Parsed page 4 of 8 for Data scientist
Parsed page 5 of 8 for Data scientist
Parsed page 6 of 8 for Data scientist
Parsed page 7 of 8 for Dat

(1 балл) Посчитайте сколько всего найдено вакансий:

In [ ]:
total=len(data)
print(total)

3362


(1 балл) Создайте структуру данных, которая будет хранить в себе имя компании и ее координаты (широта и долгота):

In [ ]:
class Company:
    name:str
    coords:tuple[float,float]|None
    def __init__(self,name,coords):
      self.name=name
      self.coords=coords


(1 балл) Создайте экземпляр этой структуры данных для каждой компании, которая выставила вакансию:

In [ ]:
companies:list[Company]=[]
for h in data:
  name=h["employer"]["name"]
  lat=None
  lon=None
  if "address" in h and h["address"] is not None and "lat" in h["address"] and "lng" in h["address"]:
    lat=h["address"]["lat"]
    lon=h["address"]["lng"]
  companies.append(Company(name,[lat,lon] if lat!=None else None))

print(len([h for h in companies if h.coords!=None]))

1664


(1 балл) Найдите количество вакансий, у которых заданы координаты (значения данных может быть пустым):

In [ ]:
companies=[h for h in companies if h.coords!=None]
print(len(companies))

1664


(1 балл) Отберите вакансии, которые расположены в пределах МКАД (можно грубо):

In [ ]:
companies_filtered=companies

(1 балл) Постройте график в координатах "широта-долгота" для отображения вакансий внутри МКАД (не забывайте прописать прозрачность, чтобы увидеть скопления):

In [ ]:
lons=[h.coords[1] for h in companies_filtered]
lats=[h.coords[0] for h in companies_filtered]

import plotly.express as px

fig=px.scatter(None,lons,lats,range_x=[37.3,37.95],range_y=[55.5,55.95],opacity=0.15)
fig.update_layout(
    xaxis=dict(
        scaleanchor='y',
        scaleratio=1,
    ),
    yaxis=dict(
        scaleanchor='x',
        scaleratio=1,
    )
)


fig.show()


(1 балл) Нанесите все точки вакансий на карту (для этого вам поможет данная <a target="_blank" href="https://plotly.com/python/mapbox-layers/">документации</a>, не забудьте про прозрачность и центрирование по Москве):

In [ ]:
lons=[h.coords[1] for h in companies_filtered]
lats=[h.coords[0] for h in companies_filtered]
moscow_center={"lat":55.751244,"lon":37.618423}

import plotly.express as px

fig = px.scatter_mapbox(lat=lats,lon=lons,opacity=0.25,center=moscow_center,zoom=9.6)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(height=900,width=1500)

fig.show()

(2 балла) **Сделайте выводы о проделанной работе:**

>*Подсказки для маленьких: Что вы можете сказать про районы Москвы? Что вы можете сказать про зарплаты? Что вы можете сказать про спрос?*

Спрос на IT-специальности в Москве традиционно остается высоким. Большинство вакансий объяснимо сосредоточено в центре города (в пределах ТТК), за его пределами следует особо отметить лишь Савеловский район и окрестности (повышенная деловая активность может объясняться некомфортностью района для проживания в связи с высокой транспортной активностью), а также кластеры с наиболее высоким в городе спросом на IT-специалистов в Москве-Сити и Технопарке

## Задание 2. Анализ данных - 20 баллов

Вам предстоит проанализировать датасет, содержащий информацию об бриллианта, их стоимости и остальных характеристиках.



Поясним значения хранящиеся в колонках:
*   `cut` - огранка бриллианта (относится к одной из примерно 10 наиболее распространенных)
*   `color` - цвет бриллианта, прозрачные бриллианты имеют градацию `D-Z`, "более высокие" буквы - более желтоватые, но часто имеют лучшие значения, поскольку цвет трудно определить один раз в кольце
*   `clarity` - чистота бриллианта, наличие включений (внутренних дефектов), чем меньше и мельче, тем лучше
*   `carat_weight` - масса бриллианта (слабо связана с размером бриллианта, т.к. огранка и качество огранки, как правило, играют большую роль)
*   `cut_quality` - качество ограники по системе оценки GIA Cut
*   `lab` - лаборатория оценивания
*   `meas_length` - длина бриллианта
*   `meas_width` - ширина бриллианта
*   `meas_depth` - глубина бриллианта
*   `total_sales_price` - цена бриллианта в долларах.
*    **и прочие колонки**

Скачаем данные:

In [ ]:
!gdown 10OJ30qlkE-7zK4JuVTDMY3U4nRB4tu_8

Downloading...
From: https://drive.google.com/uc?id=10OJ30qlkE-7zK4JuVTDMY3U4nRB4tu_8
To: /content/diamonds.csv
100% 35.4M/35.4M [00:00<00:00, 114MB/s]


Создадим DataFrame:

In [ ]:
import pandas as pd

diamonds = pd.read_csv("diamonds.csv")
diamonds

,Unnamed: 0,cut,color,clarity,carat_weight,cut_quality,lab,symmetry,polish,eye_clean,...,meas_depth,girdle_min,girdle_max,fluor_color,fluor_intensity,fancy_color_dominant_color,fancy_color_secondary_color,fancy_color_overtone,fancy_color_intensity,total_sales_price
0,0,Round,E,VVS2,0.09,Excellent,IGI,Very Good,Very Good,unknown,...,1.79,M,M,unknown,NaN,unknown,unknown,unknown,unknown,200
1,1,Round,E,VVS2,0.09,Very Good,IGI,Very Good,Very Good,unknown,...,1.78,STK,STK,unknown,NaN,unknown,unknown,unknown,unknown,200
2,2,Round,E,VVS2,0.09,Excellent,IGI,Very Good,Very Good,unknown,...,1.77,TN,M,unknown,NaN,unknown,unknown,unknown,unknown,200
3,3,Round,E,VVS2,0.09,Excellent,IGI,Very Good,Very Good,unknown,...,1.78,M,STK,unknown,NaN,unknown,unknown,unknown,unknown,200
4,4,Round,E,VVS2,0.09,Very Good,IGI,Very Good,Excellent,unknown,...,1.82,STK,STK,unknown,NaN,unknown,unknown,unknown,unknown,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219698,219699,Round,E,VS1,10.65,Excellent,GIA,Excellent,Excellent,unknown,...,8.66,M,STK,unknown,NaN,unknown,unknown,unknown,unknown,1210692
219699,219700,Radiant,unknown,VS2,5.17,unknown,GIA,Very Good,Very Good,unknown,...,5.71,TK,XTK,unknown,NaN,Green,unknown,NaN,Fancy Light,1292500
219700,219701,Round,E,VS1,18.07,Excellent,GIA,Excellent,Excellent,unknown,...,10.20,TN,M,unknown,NaN,unknown,unknown,unknown,unknown,1315496
219701,219702,Princess,unknown,SI2,0.90,unknown,GIA,Good,Good,unknown,...,3.47,XTN,VTK,unknown,Faint,Red,unknown,unknown,Fancy,1350000


(0.5 балла) Выведете первые 5 строк датасета:

In [ ]:
diamonds[:5]

,Unnamed: 0,cut,color,clarity,carat_weight,cut_quality,lab,symmetry,polish,eye_clean,...,meas_depth,girdle_min,girdle_max,fluor_color,fluor_intensity,fancy_color_dominant_color,fancy_color_secondary_color,fancy_color_overtone,fancy_color_intensity,total_sales_price
0,0,Round,E,VVS2,0.09,Excellent,IGI,Very Good,Very Good,unknown,...,1.79,M,M,unknown,NaN,unknown,unknown,unknown,unknown,200
1,1,Round,E,VVS2,0.09,Very Good,IGI,Very Good,Very Good,unknown,...,1.78,STK,STK,unknown,NaN,unknown,unknown,unknown,unknown,200
2,2,Round,E,VVS2,0.09,Excellent,IGI,Very Good,Very Good,unknown,...,1.77,TN,M,unknown,NaN,unknown,unknown,unknown,unknown,200
3,3,Round,E,VVS2,0.09,Excellent,IGI,Very Good,Very Good,unknown,...,1.78,M,STK,unknown,NaN,unknown,unknown,unknown,unknown,200
4,4,Round,E,VVS2,0.09,Very Good,IGI,Very Good,Excellent,unknown,...,1.82,STK,STK,unknown,NaN,unknown,unknown,unknown,unknown,200


(0.5 балла) Найдите количество бриллиантов всего в данном датасете:

In [ ]:
print(len(diamonds))

219703


(0.5 балла) Выведите максимальную и минимальную цену бриллианта в формате `номер - цена`:

In [ ]:
#print(diamonds.loc[0])


mx=diamonds["total_sales_price"].idxmax()
mn=diamonds["total_sales_price"].idxmin()

print(f"{mx} - {diamonds.loc[mx]['total_sales_price']}")
print(f"{mn} - {diamonds.loc[mn]['total_sales_price']}")

219702 - 1449881
0 - 200


(1 балл) Посчитайте количество пропусков (пропуском считается значение `unknown` или `None`):

In [ ]:
diamonds.isin(['unknown',None]).sum().sum()


1746027

(1 балл) Найдите в каких столбцах присутствуют пропуски:

In [ ]:
sums=diamonds.isin(['unknown',None]).sum(axis=0)
print([h for h in sums[sums>0].index])

['color', 'cut_quality', 'eye_clean', 'culet_size', 'culet_condition', 'girdle_min', 'girdle_max', 'fluor_color', 'fluor_intensity', 'fancy_color_dominant_color', 'fancy_color_secondary_color', 'fancy_color_overtone', 'fancy_color_intensity']


В нашем датасете присутствуют колонки, которые начинаются с `fancy_color_` или `fluor_`. Большинство значений в этих колонках - `unknown` или `None`, поэтому мы считаем их малоинформативными.

Ваша задача:

1. (0.5 балла) найти количество таких колонок, которые начинаются с `fancy_color_` или `fluor_`:

In [ ]:
cols_to_remove={h for h in diamonds.columns if h.startswith('fancy_color_') or h.startswith('fluor_')}
len(cols_to_remove)

6

2. (0.5 балла) найти количество информативных (не `None` или `unknown`) значений в этих колонках:

In [ ]:
mod_df=diamonds[list(cols_to_remove)]

(~mod_df.isin(["unknown",None])).sum().sum()

256727

3. (0.5 балла) удалить данные колонки из датасета, сохранив изменения (в дальнейшем работаем измененным датасетом):

In [ ]:
diamonds=diamonds[list(set(diamonds.columns)-cols_to_remove)]


(1 балл) Проделайте тоже самое для колонок, начинающихся с `culet_`:

In [ ]:
diamonds=diamonds[[h for h in diamonds.columns if not h.startswith('culet_')]]


(1 балл) Посчитайте сколько бы стоил (в тысячах долларов) сейф, в котором хранились бы все бриллианты из датасета:

In [ ]:
diamonds['total_sales_price'].sum()/1000

1517721.991

Немного необоснованных манипуляций.

(1 балл) Выберите из датасета строки с четными индексами и выведете значения (не меняя нумерацию индексов):

In [ ]:
diamonds.loc[::2]

,meas_width,meas_depth,Unnamed: 0,eye_clean,cut_quality,cut,color,girdle_min,symmetry,table_percent,lab,clarity,polish,carat_weight,total_sales_price,girdle_max,meas_length,depth_percent
0,2.87,1.79,0,unknown,Excellent,Round,E,M,Very Good,59.0,IGI,VVS2,Very Good,0.09,200,M,2.85,62.7
2,2.90,1.77,2,unknown,Excellent,Round,E,TN,Very Good,59.0,IGI,VVS2,Very Good,0.09,200,M,2.88,61.1
4,2.83,1.82,4,unknown,Very Good,Round,E,STK,Very Good,58.5,IGI,VVS2,Excellent,0.09,200,STK,2.79,64.9
6,2.88,1.84,6,unknown,Very Good,Round,E,STK,Very Good,57.0,IGI,VVS2,Very Good,0.09,200,STK,2.85,64.0
8,2.92,1.85,8,unknown,Very Good,Round,E,STK,Very Good,59.5,IGI,VVS2,Very Good,0.09,200,TK,2.89,63.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219694,14.91,9.24,219695,unknown,Excellent,Round,D,unknown,Excellent,58.0,HRD,IF,Excellent,12.52,1121792,unknown,14.84,62.1
219696,15.23,8.69,219697,unknown,unknown,Oval,H,TN,Excellent,55.0,GIA,VS2,Excellent,18.88,1132347,STK,21.34,57.1
219698,14.18,8.66,219699,unknown,Excellent,Round,E,M,Excellent,58.0,GIA,VS1,Excellent,10.65,1210692,STK,14.06,61.3
219700,17.10,10.20,219701,unknown,Excellent,Round,E,TN,Excellent,59.0,GIA,VS1,Excellent,18.07,1315496,M,17.06,60.2


(0.5 балла) А теперь с индексами, кратными 6:

In [ ]:
diamonds.loc[::6]

,meas_depth,girdle_min,clarity,cut,table_percent,carat_weight,Unnamed: 0,polish,eye_clean,meas_length,depth_percent,meas_width,cut_quality,color,total_sales_price,lab,girdle_max,symmetry
0,1.79,M,VVS2,Round,59.0,0.09,0,Very Good,unknown,2.85,62.7,2.87,Excellent,E,200,IGI,M,Very Good
6,1.84,STK,VVS2,Round,57.0,0.09,6,Very Good,unknown,2.85,64.0,2.88,Very Good,E,200,IGI,STK,Very Good
12,1.77,STK,VVS2,Round,58.5,0.09,12,Very Good,unknown,2.85,61.8,2.89,Very Good,E,200,IGI,TK,Very Good
18,1.78,M,VVS2,Round,60.0,0.09,18,Very Good,unknown,2.91,61.0,2.92,Excellent,E,200,IGI,M,Very Good
24,0.00,unknown,I1,Round,0.0,0.34,24,Excellent,unknown,0.00,0.0,0.00,Excellent,L,204,IGI,unknown,Excellent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219678,8.69,TN,VS1,Round,55.0,10.23,219679,Excellent,unknown,13.82,62.7,13.92,Excellent,G,859320,GIA,M,Excellent
219684,3.16,TK,I1,Radiant,57.0,0.89,219685,Good,unknown,6.44,63.8,4.96,unknown,unknown,982560,GIA,VTK,Good
219690,8.39,M,VVS2,Round,59.0,10.10,219691,Excellent,unknown,14.07,59.4,14.15,Excellent,D,1040739,GIA,STK,Excellent
219696,8.69,TN,VS2,Oval,55.0,18.88,219697,Excellent,unknown,21.34,57.1,15.23,unknown,H,1132347,GIA,STK,Excellent


(1 балл) Выберете из датасета только категорильные признаки и выведете датасет:

In [ ]:
diamonds.select_dtypes(include=[object])

,girdle_min,clarity,cut,polish,eye_clean,cut_quality,color,lab,girdle_max,symmetry
0,M,VVS2,Round,Very Good,unknown,Excellent,E,IGI,M,Very Good
1,STK,VVS2,Round,Very Good,unknown,Very Good,E,IGI,STK,Very Good
2,TN,VVS2,Round,Very Good,unknown,Excellent,E,IGI,M,Very Good
3,M,VVS2,Round,Very Good,unknown,Excellent,E,IGI,STK,Very Good
4,STK,VVS2,Round,Excellent,unknown,Very Good,E,IGI,STK,Very Good
...,...,...,...,...,...,...,...,...,...,...
219698,M,VS1,Round,Excellent,unknown,Excellent,E,GIA,STK,Excellent
219699,TK,VS2,Radiant,Very Good,unknown,unknown,unknown,GIA,XTK,Very Good
219700,TN,VS1,Round,Excellent,unknown,Excellent,E,GIA,M,Excellent
219701,XTN,SI2,Princess,Good,unknown,unknown,unknown,GIA,VTK,Good


>**Примечание**
>
>Данный датасет можно использовать для задания 3 (с использованием `sklearn`)

(0.5 балла) Выберете столбец с данными о цене бриллиантов и домножите его на тот столбец, только транспонированный, и выведите полученную матрицу:

In [ ]:
import numpy as np
# Считаю, что нужно умножить строку на столбец, иначе матрица слишком большая
prices=diamonds["total_sales_price"].to_numpy()
prices.reshape(1,-1).dot(prices.reshape(-1,1))

array([[158540672665477]])

### Построение графиков

**Требования к графику:**

1.   График должен быть подписан
2.   Оси должны быть подписаны
3.   Должна быть легенда графика
4.   График должен быть информативным (желательно без сильных выбросов)

**Все надписи на русском языке!**

>**Примечание**
>
>График необходимо строить для очищенных данных - в них не должно быть пропусков по осям, по которым строите график. При этом **не перезатирайте** имеющийся датафрейм!

(2 балла) Постройте график `цвет бриллианта - стоимость`:

In [ ]:
# your code here

(2 балла) Постройте график `лабараторная оценка - кол-во бриллиантов`:

In [ ]:
# your code here

(2 балла) Постройте график `вес бриллианта - стоимость`:

In [ ]:
# your code here

(2 балла) Постройте график `система оценки GIA Cut - кол-во бриллиантов`:

In [ ]:
# your code here

(2 балла) **Сделайте выводы по построенным графикам:**

>*Подсказка для маленьких: Что вы можете сказать о графиках полученных выше для данного датасета?*

Ваши выводы здесь :)

## Задание 3. Модель - 30 баллов

В данном задании вам предстоит построить классифицирующую модель размер алмаза и его стоимости, а затем оценить полученную модель.

>**Примечание**
>
>Данная модель не претендует на самый верный индикатор стоимости алмаза, ведь мы не учитываем ничего больше чем стоимость алмаза. Исключительно для образовательных целей :)

Для построения модели нам понадобится 4 столбца, а именно:
- `meas_length`
- `meas_width`
- `meas_depth`
- `total_sales_price`

От остальных столбцов можно избавиться.

(1 балл) Создайте новый датасет и выведите его первые 10 строк:

In [ ]:
data=diamonds[['meas_length','meas_width','meas_depth','total_sales_price']]
data[:10]

,meas_length,meas_width,meas_depth,total_sales_price
0,2.85,2.87,1.79,200
1,2.84,2.89,1.78,200
2,2.88,2.90,1.77,200
3,2.86,2.88,1.78,200
4,2.79,2.83,1.82,200
5,2.95,2.99,1.81,200
6,2.85,2.88,1.84,200
7,2.86,2.89,1.78,200
8,2.89,2.92,1.85,200
9,2.83,2.87,1.80,200


(1 балл) Перемешайте строки в датафрейме случайным образом:

In [ ]:
data=data.sample(frac=1)


,meas_length,meas_width,meas_depth,total_sales_price
115228,5.45,5.47,3.37,2240
104434,4.78,4.79,2.88,1770
77044,4.68,4.71,2.94,1120
41798,4.54,3.44,2.34,886
129981,5.43,5.45,3.44,2868
...,...,...,...,...
196950,7.55,5.53,3.74,13905
15821,3.89,3.92,2.45,678
78869,4.27,4.30,2.67,1142
31961,4.33,4.37,2.72,838


(1 балл) Сформируем новую колонку - для каждого бриллианта найдем сумму Минковского (p=3) по 3 измерениям бриллианта:

In [ ]:
def minc(a,b,c):
  return ((a**3+b**3+c**3))**(1/3)

data["metric"]=minc(data["meas_length"],data["meas_width"],data["meas_depth"])
data

,meas_length,meas_width,meas_depth,total_sales_price,metric
115228,5.45,5.47,3.37,2240,7.138851
104434,4.78,4.79,2.88,1770,6.240298
77044,4.68,4.71,2.94,1120,6.148185
41798,4.54,3.44,2.34,886,5.278794
129981,5.43,5.45,3.44,2868,7.131456
...,...,...,...,...,...
196950,7.55,5.53,3.74,13905,8.670357
15821,3.89,3.92,2.45,678,5.114763
78869,4.27,4.30,2.67,1142,5.608275
31961,4.33,4.37,2.72,838,5.695552


(1 балл) Сформируем новый датасет из двух колонок - сумма Минковского и `total_sales_price`:

In [ ]:
data2=data[["metric","total_sales_price"]]
data2

,metric,total_sales_price
115228,7.138851,2240
104434,6.240298,1770
77044,6.148185,1120
41798,5.278794,886
129981,7.131456,2868
...,...,...
196950,8.670357,13905
15821,5.114763,678
78869,5.608275,1142
31961,5.695552,838


(1 балл) Разделите вашу выборку на две части в соотношении 8:2 - одна выборка будет для тренировка, а другая для тестов вашей модели:

In [ ]:
border=int(0.8*len(data2))
train=data.iloc[:border]
test=data2.loc[border:]

(2 балла) Постройте график зависимости стоимости от вашего нового параметра размера для тренинг-выборки:

(2 балла) Нанесите на тот же график тестовую выборку, используя другой цвет (не забудьте про параметр прозрачности):

In [ ]:
# your code here

(3 балла) Найдите для каждой бриллианта из тестовой выборки его "ближайшего соседа" из тренинг-выборки. Для каждого бриллианта из тестовой выборки найдите расстояния до соответствующего ближайшего соседа:

In [ ]:
# your code here

(3 балла) Найдите насколько сильно (в процентах) отличается цена бриллиантов из тестовой выборки от их соответствующих ближайших соседей:

In [ ]:
# your code here

**Данная модель является выдумкой автора. Не важен коэффициент ошибки, важны ваши шаги для получения коэффициента.**

### Введение в машинное обучение

Теперь давайте познакомимся с таким сильным инструментом как `sklearn`.

Вам необходимо:
1. выбрать 2 категориальных признака
2. на их основе обучить решающее дерево
3. оценить среднеквадратичную ошибку цены бриллианта
4. сделать пп. 1-3 и выбрать ту пару, которая дает лучший результат
5. сделать выводы

(3 балла) Мы работаем с категориальными признаками, для них не понятно что есть "больше" и "меньше". Поэтому нам нужно предварительно закодировать значения наших признаков, с чем нам поможет [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):

,meas_length,meas_width,meas_depth,total_sales_price,metric
115228,5.45,5.47,3.37,2240,7.138851
104434,4.78,4.79,2.88,1770,6.240298
77044,4.68,4.71,2.94,1120,6.148185
41798,4.54,3.44,2.34,886,5.278794
129981,5.43,5.45,3.44,2868,7.131456
...,...,...,...,...,...
196950,7.55,5.53,3.74,13905,8.670357
15821,3.89,3.92,2.45,678,5.114763
78869,4.27,4.30,2.67,1142,5.608275
31961,4.33,4.37,2.72,838,5.695552


(3 балла) Разбейте датасет на обучающую и тестовую выбору. Для этого вам поможет [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html):

In [ ]:
# your code here

(3 балла) Затем создайте и обучите вашу модель - [DecisionTreeRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html):

In [ ]:
# your code here

(3 балла) Оцените ошибку вашей модели при помощи [mean_squared_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html):

In [ ]:
# your code here

(3 балла) **Сделайте выводы какие категориальные признаки лучше:**

*Подсказки для маленьких: Что вы можете сказать о зависимости по вашей выборке признаков? Какой вы можете сделать вывод об готовых инструментах для работы и написанных Вами?*

Ваши выводы здесь :)